In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('hh_ru_dataset.csv')

In [2]:
salary_df = df.copy()
salary_df = salary_df[~salary_df.compensation_to.isna()]
salary_df['final'] = salary_df['final_state'].map(lambda x: int(x == 'invitation'))

In [11]:
a = salary_df[salary_df.expected_salary > salary_df.compensation_to].final
b = salary_df[salary_df.expected_salary <= salary_df.compensation_to].final
# diff = b - a
minuses = a.sum() * (len(b) - b.sum())
zeros = a.sum() * b.sum() + (len(a) - a.sum()) * (len(b) - b.sum())
pluses = (len(a) - a.sum()) * b.sum()
mean = (pluses-minuses)/(pluses+zeros+minuses)
se = (minuses * ((-1 - mean)**2) + zeros*(mean**2) + pluses*((1-mean) ** 2)) / (pluses+zeros+minuses)
dispn = (se/(pluses+zeros+minuses))**0.5
mean, dispn

(0.10682999997194546, 3.960351671475557e-06)

по-хорошему определить какой-то интервал, когда ожидаемая зарплата не сильно отклоняется от отрезка зарплат закачика. при этом этот инетрвал может зависет от длины отрезка зп заказчика

In [9]:
def find_info(group):
    a = group[group.expected_salary > group.compensation_to].final
    b = group[group.expected_salary <= group.compensation_to].final
    print(f"Доля успехов в группе с завышенной зп: {a.mean()}")
    print(f"Доля успехов в группе с нормальной зп: {b.mean()}")
    diff = b.mean() - a.mean()
    print(f"Разница между долями: {diff}")
    print(f"Стандартное отклонение в датасете: {group.final.std()}")
    print(f"Размер датасета: {len(group)}")
    t = (b.mean() - a.mean()) / (((a.std() ** 2) / len(a) + (b.std() ** 2) / len(b)) ** 0.5)
    print(f"t-критерий для групп: {t}")
    p = (mean - diff) / dispn
    print(f"p-value={p}")
    return (a.mean(), b.mean(), len(group), diff, group.final.std(), t)

In [10]:
find_info(salary_df[salary_df.work_experience_months > 60])

Доля успехов в группе с завышенной зп: 0.25659466359947924
Доля успехов в группе с нормальной зп: 0.3543738521448222
Разница между долями: 0.09777918854534295
Стандартное отклонение в датасете: 0.4650593223378044
Размер датасета: 215278
t-критерий для групп: 47.97120851033458
p-value=2285.355487945932


(0.25659466359947924,
 0.3543738521448222,
 215278,
 0.09777918854534295,
 0.4650593223378044,
 47.97120851033458)

Можно посмотреть эту diff'у относительно всей совокупности

In [15]:
df[['initial_state', 'final_state']].value_counts() / len(df)

initial_state  final_state
response       response       0.485478
               invitation     0.205952
invitation     invitation     0.161046
response       discard        0.144012
invitation     discard        0.001692
discard        discard        0.000152
dtype: float64

In [19]:
df.education_level.value_counts()

higher               230113
special_secondary    108860
secondary             69761
unfinished_higher     48194
bachelor              28443
master                12924
candidate              1543
doctor                   60
Name: education_level, dtype: int64